In [1]:
from tqdm import tqdm

import torch
from torch import nn
from torch.utils.data import DataLoader

from data import DrawingDataset

# from mamba import Mamba, MambaConfig
from mamba_ssm import Mamba

/home/nicbat/miniconda3/envs/drawenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset = DrawingDataset(data_path="./data", split="train", max_length=100)
val_dataset = DrawingDataset(data_path="./data", split="valid", max_length=100)
test_dataset = DrawingDataset(data_path="./data", split="test", max_length=100)

train = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val = DataLoader(dataset=val_dataset, batch_size=64, shuffle=True)
test = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)

100%|██████████| 345/345 [00:06<00:00, 56.32it/s]


In [3]:
print(f"Length of train dataloader: {len(train)}")
print(f"Length of validation dataloader: {len(val)}")
print(f"Length of test dataloader: {len(test)}")

for i,data in tqdm(enumerate(train)):
    sketch, label = data
    if i == 0:
        print(sketch.shape, len(label))
        print(sketch[0])
    break

Length of train dataloader: 351999
Length of validation dataloader: 12567
Length of test dataloader: 12574


0it [00:00, ?it/s]

0it [00:01, ?it/s]

torch.Size([64, 100, 5]) 64
tensor([[   8.,    1.,    1.,    0.,    0.],
        [  15.,  -10.,    1.,    0.,    0.],
        [  12.,  -15.,    1.,    0.,    0.],
        [   2.,  -62.,    1.,    0.,    0.],
        [  -7.,  -18.,    1.,    0.,    0.],
        [ -14.,  -16.,    1.,    0.,    0.],
        [ -28.,  -13.,    1.,    0.,    0.],
        [ -36.,    1.,    1.,    0.,    0.],
        [ -15.,   10.,    1.,    0.,    0.],
        [ -16.,   28.,    1.,    0.,    0.],
        [ -10.,   34.,    1.,    0.,    0.],
        [  -7.,   12.,    1.,    0.,    0.],
        [  -6.,    4.,    1.,    0.,    0.],
        [  -5.,    2.,    1.,    0.,    0.],
        [ -41.,   -2.,    1.,    0.,    0.],
        [ -30.,   15.,    1.,    0.,    0.],
        [  -4.,   22.,    1.,    0.,    0.],
        [   3.,    6.,    1.,    0.,    0.],
        [  15.,    7.,    1.,    0.,    0.],
        [ 142.,   -1.,    1.,    0.,    0.],
        [  11.,   -2.,    1.,    0.,    0.],
        [  15.,   -8.,    0

In [4]:
# inference = nn.Sequential(
#     nn.RNN(5, 100, 100, 1),
#     # nn.Linear(100, 345),
# )

# lr = 0.001
# epochs = 5
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(inference.parameters(), lr=lr)
# for epoch in range(epochs):
#     for i, data in enumerate(train):
#         sketch, label = data
#         print(sketch.shape)
#         optimizer.zero_grad()
#         output = inference(sketch)
#         loss = criterion(output, label)
#         loss.backward()
#         optimizer.step()
#         print(f"Epoch: {epoch}, Loss: {loss.item()}")

In [5]:
print(sketch.shape, label)
print(sketch[0][0].shape)
# model = nn.Sequential(
# )
# reshaped = torch.reshape(sketch, (64, 5, 100))
# print(reshaped.shape)
# print(reshaped[0])
model = Mamba(
        d_model = 5,
        d_state = 16,
        d_conv = 4,
        expand=2
    ).to("cuda")

# print(sketch.dtype, sketch[0].size())
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
test = model(sketch.to("cuda"))
print(test.shape)
print(test[0][0])
# (B,L,D) - (64, 5, ???????????)

torch.Size([64, 100, 5]) ('dog', 'barn', 'bulldozer', 'floor lamp', 'pillow', 'snail', 'mermaid', 'peas', 'rainbow', 'postcard', 'sword', 'train', 'The Mona Lisa', 'scissors', 'hedgehog', 'bathtub', 'chair', 'ear', 'marker', 'birthday cake', 'lollipop', 'owl', 'eyeglasses', 'mosquito', 'waterslide', 'lantern', 'sword', 'triangle', 'swan', 'teddy-bear', 'campfire', 'microwave', 'flip flops', 'donut', 'sandwich', 'foot', 'rollerskates', 'postcard', 'chandelier', 'sink', 'postcard', 'van', 'nail', 'violin', 'van', 'necklace', 'sock', 'crown', 'pickup truck', 'octopus', 'harp', 'passport', 'knife', 'mouse', 'crocodile', 'speedboat', 'rabbit', 'octopus', 'parachute', 'stove', 'bucket', 'pond', 'crown', 'golf club')
torch.Size([5])
torch.Size([64, 100, 5])
tensor([-0.4441,  0.4422, -0.0507, -0.2075, -0.5100], device='cuda:0',
       grad_fn=<SelectBackward0>)
